### Exercise 1
Suppose the people who own page 3 in the web of Figure 1 are infuriated by the
fact that its importance score, computed using formula (2.1), is lower than the score of page 1. In
an attempt to boost page 3’s score, they create a page 5 that links to page 3; page 3 also links to page
5. Does this boost page 3’s score above that of page 1?

![image](./image.png)

In [7]:
import numpy as np

A = np.array([[0, 0, 1, 1/2],
         [1/3, 0, 0, 0],
         [1/3, 1/2, 0, 1/2],
         [1/3, 1/2, 0, 0]])

print(A)

[[0.         0.         1.         0.5       ]
 [0.33333333 0.         0.         0.        ]
 [0.33333333 0.5        0.         0.5       ]
 [0.33333333 0.5        0.         0.        ]]


In [8]:
for i in range(len(A)):
    print(sum(A.T[i]))
    

1.0
1.0
1.0
1.0


In [11]:
np.linalg.eig(A)

EigResult(eigenvalues=array([ 1.        +0.j        , -0.36062333+0.41097555j,
       -0.36062333-0.41097555j, -0.27875333+0.j        ]), eigenvectors=array([[ 0.72101012+0.j        , -0.75521571+0.j        ,
        -0.75521571-0.j        ,  0.50648562+0.j        ],
       [ 0.24033671+0.j        ,  0.3036721 +0.34607247j,
         0.3036721 -0.34607247j, -0.60565568+0.j        ],
       [ 0.54075759+0.j        ,  0.09315321-0.2746779j ,
         0.09315321+0.2746779j , -0.38153917+0.j        ],
       [ 0.36050506+0.j        ,  0.3583904 -0.07139457j,
         0.3583904 +0.07139457j,  0.48070923+0.j        ]]))

In [12]:

def my_gauss(M, tol=1e-8):
    """
    Function that compute the row echelon form of a given matrix M.
    :param M: 2D-array object (numpy ndarray)
    :param tol: tolerance for setting to zero the numerical zeros for computing Around
    :return A: 2D-array object (numpy ndarray) representing the row echelon form of M
    :return Around: The matrix A where values under the tolerance are set to zeros (to obtain a "real" row echelon form)
    :return rank: return the rank of M (computed exploiting the row echelon form A)
    """

    A = M.copy()
    m, n = A.shape

    k = 0  # pivot index
    j = 0  # column index
    
    # ATTENTION!: remember the 0-indexing convention of Python!
    while k < m and j < n:
        # rows corresponding to max abs in columns
        row_maxabsel = k + np.abs(A[k:, j]).argmax(axis=0)
        
        # max abs values in column
        maxabsel = A[row_maxabsel, j]

        # Swap "maximum-abs-row" (row_maxabsel) with row k and
        # perform operations to put as 0 all the entries below entry of position (k,j)
        if ... :  # <-- TODO! (insert the check on maxabsel not zero... using the tolerance!)
            # Swapping
            kth_row = A[k, :].copy()
            ... # <-- TODO!
            # write above one or more lines of code to complete the swap between rows.
            
            # Zeros: DO NOT USE A for!!!
            factor_vec = np.array(A[(k + 1):, j] / maxabsel, ndmin=2).T
            ... # <-- TODO!
            # complete the line above to build a column vector with the coefficients a_ij/maxabsel 
            # necessary to set to zero the elements under the pivot.
            
            k += 1

        j += 1

    Around = A.copy()
    # For aesthetic reasons only:
    Around[np.abs(A) < tol] = 0
    
    # RANK COMPUTATION
    smallest_axis = np.argmin(Around.shape)  # case m == n --> returns 0 ('rows')
    nonzero_axis = Around.nonzero()[smallest_axis]
    rank = np.unique(nonzero_axis).size

    return A, Around, rank

In [13]:
my_gauss(A)

/var/folders/gw/h_3h_b5d74v_ds3nx7nhw4hh0000gn/T/ipykernel_13153/1401955892.py:34: RuntimeWarning: invalid value encountered in divide
  factor_vec = np.array(A[(k + 1):, j] / maxabsel, ndmin=2).T


(array([[0.        , 0.        , 1.        , 0.5       ],
        [0.33333333, 0.        , 0.        , 0.        ],
        [0.33333333, 0.5       , 0.        , 0.5       ],
        [0.33333333, 0.5       , 0.        , 0.        ]]),
 array([[0.        , 0.        , 1.        , 0.5       ],
        [0.33333333, 0.        , 0.        , 0.        ],
        [0.33333333, 0.5       , 0.        , 0.5       ],
        [0.33333333, 0.5       , 0.        , 0.        ]]),
 4)